In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from supervised.automl import AutoML

In [2]:
df_init = pd.read_csv('data_v1.csv', sep=';')
df_init.head()

,Id,Number,Result,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_106,...,Feature_99,Feature_100,Feature_111,Feature_1122,Feature_112,Feature_113,Feature_114,Feature_115,Feature_116,Feature_117
0,1,10359/C2020,2,2,56,12,1,7.0,4,2,...,0.0,0.0,1,0,1,1,1.0,1,1,0
1,2,10346/C2020,2,2,69,19,1,6.0,4,2,...,0.0,0.0,1,0,1,1,1.0,0,1,1
2,3,10311/C2020,2,1,66,8,1,4.0,4,2,...,0.0,0.0,1,0,1,1,1.0,0,1,0
3,4,10292/C2020,2,2,62,16,1,NaN,3,2,...,0.0,0.0,1,0,1,1,0.0,0,1,1
4,5,10283/C2020,2,2,67,30,1,NaN,4,2,...,0.0,0.0,1,0,1,1,1.0,0,1,0


In [4]:
target_feature = 'Result'
features = ['Feature_63', 'Feature_51', 'Feature_31', 'Feature_47', 'Feature_41', 'Feature_70', 'Feature_6', 'Feature_40', 'Feature_2',
            'Feature_49', 'Feature_59', 'Feature_46', 'Feature_73', 'Feature_71', 'Feature_9', 'Feature_28', 'Feature_72', 'Feature_48',
            'Feature_37', 'Feature_65']

df_ = df_init[[target_feature] + features]

num_features = ['Feature_2', 'Feature_3', 'Feature_33', 'Feature_34', 'Feature_36', 'Feature_37', 'Feature_38', 'Feature_39', 'Feature_40', 
                'Feature_41', 'Feature_42', 'Feature_43', 'Feature_44', 'Feature_45', 'Feature_46', 'Feature_47', 
                'Feature_48', 'Feature_49', 'Feature_50', 'Feature_51', 'Feature_53', 'Feature_55', 'Feature_57', 
                'Feature_58', 'Feature_59', 'Feature_64', 'Feature_70', 'Feature_71', 'Feature_72', 'Feature_73']

special_features = ['Feature_17', 'Feature_18', 'Feature_23', 'Feature_74', 'Feature_75', 'Feature_76', 'Feature_77', 
                    'Feature_78', 'Feature_79', 'Feature_80', 'Feature_81', 'Feature_82', 'Feature_83', 'Feature_84', 
                    'Feature_85', 'Feature_86', 'Feature_87', 'Feature_88', 'Feature_89', 'Feature_90', 'Feature_91', 
                    'Feature_92', 'Feature_93', 'Feature_94', 'Feature_95', 'Feature_96', 'Feature_97', 'Feature_98', 
                    'Feature_99', 'Feature_100']

for col in special_features:
    if col in df_.columns:
        df_[col].fillna(0, inplace=True)

cat_features = []
for col in df_.drop(target_feature, axis=1).columns:
    if col in num_features:
        df_[col].fillna(df_[col].median(), inplace=True)
        df_[col] = df_[col].astype('float64')
    else:
        cat_features.append(col)
        df_[col].fillna(-1, inplace=True)
        df_[col] = df_[col].astype('int64')

# Result=0 - alive, Result=1 died
df_.loc[df_[target_feature] == 1, target_feature] = 0
df_.loc[df_[target_feature] == 2, target_feature] = 1

df_train, df_test = train_test_split(
    df_,
    shuffle=True,
    test_size=0.25,
    random_state=0,
    stratify=df_[target_feature],
)

# Separate majority and minority classes
df_majority = df_train[df_train[target_feature] == 1]
df_minority = df_train[df_train[target_feature] == 0]

# Upsample majority class
df_majority_upsampled = resample(
    df_majority,
    replace=True,
    n_samples=100,
    random_state=0,
)
# Upsample minority class
df_minority_upsampled = resample(
    df_minority,
    replace=True,
    n_samples=100,
    random_state=0,
)
# Combine minority class with downsampled majority class
df_train_sampled = pd.concat([df_majority_upsampled, df_minority_upsampled])

X_train = df_train_sampled[features]
y_train = df_train_sampled[target_feature]

X_test = df_test[features]
y_test = df_test[target_feature]

automl = AutoML(
    results_path='reports/automl-ver_2/',
    mode='Perform',
    ml_task='binary_classification',
)
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a D

AutoML directory: reports/automl/
Expected computing time:
Time for tuning with Optuna: len(algorithms) * optuna_time_budget = 21600 seconds
There is no time limit for ML model training after Optuna tuning (total_time_limit parameter is ignored).
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network']
AutoML will stack models
AutoML will ensemble availabe models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 6 models


[I 2021-08-04 18:15:25,837] A new study created in memory with name: no-name-5879795f-f70c-4db0-b1c6-982695cc8de3


Optuna optimizes LightGBM with time budget 3600 seconds eval_metric logloss (minimize)


[I 2021-08-04 18:15:26,515] Trial 0 finished with value: 0.07920841082301297 and parameters: {'learning_rate': 0.1, 'num_leaves': 1598, 'lambda_l1': 2.840098794801191e-06, 'lambda_l2': 3.0773599420974e-06, 'feature_fraction': 0.8613105322932351, 'bagging_fraction': 0.970697557159987, 'bagging_freq': 7, 'min_data_in_leaf': 36, 'extra_trees': False}. Best is trial 0 with value: 0.07920841082301297.
[I 2021-08-04 18:15:27,972] Trial 1 finished with value: 0.44056367152088977 and parameters: {'learning_rate': 0.0125, 'num_leaves': 30, 'lambda_l1': 0.09024841733204539, 'lambda_l2': 0.8785585624049705, 'feature_fraction': 0.5554201923798203, 'bagging_fraction': 0.7307773310574073, 'bagging_freq': 1, 'min_data_in_leaf': 37, 'extra_trees': True}. Best is trial 0 with value: 0.07920841082301297.
[I 2021-08-04 18:15:32,839] Trial 2 finished with value: 0.22364990485394642 and parameters: {'learning_rate': 0.025, 'num_leaves': 1781, 'lambda_l1': 8.42482357544477e-05, 'lambda_l2': 0.16570239237798

KeyboardInterrupt: 